# Домашнє завдання. Аналіз даних у PySpark

In [30]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, round

### 1. Завантажте та прочитайте кожен CSV-файл як окремий DataFrame.

In [ ]:
spark = SparkSession.builder.appName("MyGoitSparkSandbox").getOrCreate()

products_df = spark.read.csv('products.csv', header=True)
purchases_df = spark.read.csv('purchases.csv', header=True)
users_df = spark.read.csv('users.csv', header=True)

### 2. Очистіть дані, видаляючи будь-які рядки з пропущеними значеннями.

In [11]:
products_df = products_df.dropna()
purchases_df = purchases_df.dropna()
users_df = users_df.dropna()

In [24]:
print(f"products_df:\n{products_df.columns}")
print(f"purchases_df:\n{purchases_df.columns}")
print(f"users_df:\n{users_df.columns}")

products_df:
['product_id', 'product_name', 'category', 'price']
purchases_df:
['purchase_id', 'user_id', 'product_id', 'date', 'quantity']
users_df:
['user_id', 'name', 'age', 'email']


### 3. Визначте загальну суму покупок за кожною категорією продуктів.

In [36]:
combined_df = purchases_df.join(products_df, on='product_id')
# print(combined_df.show())
combined_df = combined_df.withColumn('sum', col('quantity') * col('price'))
result_df = combined_df.groupBy('category').agg(round(sum('sum'),2).alias('total_sum'))

result_df.show()

+-----------+---------+
|   category|total_sum|
+-----------+---------+
|       Home|   1523.5|
|     Sports|   1802.5|
|Electronics|   1174.8|
|   Clothing|    790.3|
|     Beauty|    459.9|
+-----------+---------+



### 4. Визначте суму покупок за кожною категорією продуктів для вікової категорії від 18 до 25 включно.

In [42]:
combined_df = purchases_df.join(products_df, on='product_id').join(users_df, on='user_id').filter((col('age') >= 18) & (col('age') <= 25))
# print(combined_df.show())
combined_df = combined_df.withColumn('sum', col('quantity') * col('price'))
# print(combined_df.show())
result_df = combined_df.groupBy('category').agg(round(sum('sum'),2).alias('total_sum'))

result_df.show()

+-----------+---------+
|   category|total_sum|
+-----------+---------+
|       Home|    361.1|
|     Sports|    310.5|
|Electronics|    249.6|
|   Clothing|    245.0|
|     Beauty|     41.4|
+-----------+---------+



### 5. Визначте частку покупок за кожною категорією товарів від сумарних витрат для вікової категорії від 18 до 25 років.

In [50]:
total_sum = result_df.agg(sum('total_sum')).collect()[0][0]
# print(total_sum)
result_df = result_df.withColumn('percentage', round(col('total_sum') * 100 / total_sum, 1)).select('category', 'percentage')
result_df.show()

+-----------+----------+
|   category|percentage|
+-----------+----------+
|       Home|      29.9|
|     Sports|      25.7|
|Electronics|      20.7|
|   Clothing|      20.3|
|     Beauty|       3.4|
+-----------+----------+



### 6. Виберіть 3 категорії продуктів з найвищим відсотком витрат споживачами віком від 18 до 25 років.

In [52]:
top_categories_df = result_df.orderBy(col('percentage').desc()).limit(3)
top_categories_df.show()


+-----------+----------+
|   category|percentage|
+-----------+----------+
|       Home|      29.9|
|     Sports|      25.7|
|Electronics|      20.7|
+-----------+----------+

